<a href="https://colab.research.google.com/github/UDICatNCHU/SparkTutorial/blob/answer/(Spark_Tutorial)_%E4%B8%8A%E8%AA%B2%E8%AC%9B%E7%BE%A9_2019_Spark_DataFrame_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! wget -O init_env.sh https://www.dropbox.com/s/6bnwn8u2hz19s59/init_env.sh && \
bash init_env.sh

In [ ]:
import os, sys
os.environ['SPARK_HOME'] = "/usr/local/spark"
os.environ['PYSPARK_PYTHON'] = "/usr/local/bin/python"
sys.path.append("/usr/local/spark/python/")
sys.path.append("/usr/local/spark/python/lib/pyspark.zip")
sys.path.append("/usr/local/spark/python/lib/py4j-0.10.4-src.zip")

In [ ]:
from pyspark import SparkContext
from pyspark import SparkConf
sc =SparkContext()

In [ ]:
!wget -O pm25.csv "https://www.dropbox.com/s/zkn3ba7pitv83el/pm2.5Taiwan.csv?dl=0"

In [ ]:
weather = sc.textFile("./pm25.csv")

In [ ]:
weather_data_rdd = weather.map(lambda line : line.split(","))
# pm25schema = weather_data_rdd.first()
# print(pm25schema)

# 回想如何使用RDD計算求取2015年，大里每小時的平均pm25數值。

In [ ]:
import math
def remove_row_with_noise (x):
    for i in range(3, len(x)):
        if not x[i].isdecimal():
            return False
    return True 

def hourKeyGen(x):
    hourkeypair = []
    x=x[3:]
    for i, value in enumerate(x):
      print(i, value)
      hourkeypair.append((i, float(value)))
    return hourkeypair

clean_weather_data = weather_data_rdd\
                    .filter(lambda x: x!=pm25schema)\
                    .filter(remove_row_with_noise)

In [ ]:
dalipm25 = clean_weather_data.filter(lambda x: x[2]== "PM2.5")

In [ ]:
count = dalipm25.count()
HourSum = dalipm25\
            .flatMap(hourKeyGen)\
            .reduceByKey(lambda x,y: x+y)\
            .mapValues(lambda x: x/count)\
            .map(lambda x: (x[1],x[0])).top(24)

print(HourSum)

# 使用DataFrame 來計算每小時平均值

In [ ]:
print(dalipm25.first())

In [ ]:
from pyspark.sql import SQLContext
from pyspark.sql import Row
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .getOrCreate()

In [ ]:
for i in dalipm25.take(5):
  print(i)

In [ ]:
from pyspark.sql import SQLContext
from pyspark.sql import Row

dalipm25row = dalipm25.map(lambda p:
        Row(
        date = p[0],
        location = p[1],
        measure = p[2],
        hr_01 = float(p[3]), hr_02 = float(p[4]),hr_03 = float(p[5]),hr_04 = float(p[6]),hr_05 = float(p[7]),
        hr_06 = float(p[8]), hr_07 = float(p[9]),hr_08 = float(p[10]),hr_09 = float(p[11]),hr_10 = float(p[12]),
        hr_11 = float(p[13]),hr_12 = float(p[14]),hr_13 = float(p[15]),hr_14 = float(p[16]),hr_15 = float(p[17]),
        hr_16 = float(p[18]),hr_17 = float(p[19]),hr_18 = float(p[20]),hr_19 = float(p[21]),hr_20 = float(p[22]),
        hr_21 = float(p[23]),hr_22 = float(p[24]),hr_23 = float(p[25]),hr_24 = float(p[26]),
    )
)

dalipm25row.take(5)

In [ ]:
df = spark.createDataFrame(dalipm25row)
df.show()

In [ ]:
df.filter(df["date"]=="2015/01/22").select("hr_01", "hr_02", "location").show()

In [ ]:
df.filter(df["date"]=="2015/01/22").select("hr_01","location").show()

In [ ]:
df.describe().show()

# 直接透過rdd生成data frame. toDF()

In [ ]:
pm25schema = weather_data_rdd.first()

In [ ]:
pm25schema

In [ ]:
dalipm25.toDF().show()

In [ ]:
pm25schema

In [ ]:
df2 = dalipm25.toDF(pm25schema)

In [ ]:
df2.schema

In [ ]:
dfpm25 = dalipm25.toDF(pm25schema)

In [ ]:
dfpm25.columns

In [ ]:
for i in dfpm25.columns[3:]:
  dfpm25 = dfpm25.withColumn(i, dfpm25[i].cast("double"))

In [ ]:
dfpm25.printSchema()

# 使用 DataFrame.filter() 來進行row資料之條件計算 http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame

In [ ]:
df.filter(df.hr_01>30).show()

# 使用 DataFrame.select() 來進行資料之Projection http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame

In [ ]:
df.select("hr_01", "hr_02", "location", "measure").show()

## 使用 DataFrame.describe() 來進行DataFrame or Column 資料之統計 http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame

In [ ]:
df.describe().show()

# 使用 DataFrame.agg() 來進行column數值之統計計算 http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame

In [ ]:
df.groupBy().avg().show()

In [ ]:
from pyspark.sql import functions as F
df.agg(F.mean(df.hr_01),F.mean(df.hr_02)).show()

#使用 DataFrame.withColumn() 來進行column新增column http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame

In [ ]:
df.show()

In [ ]:
df.withColumn('hr_avg', (df.hr_01+df.hr_02)/2).show()

# 練習1: 請用SPARK SQL算算看2015全國哪個測站，紫爆天數最多？

# 練習2: 請使用SPARK SQL求取2015年，全國pm2.5最高的前十個工作站測點以及其日期。

In [51]:
df = clean_weather_data.toDF()
for i in df.columns[3:]:
  df = df.withColumn(i, df[i].cast("double"))

In [53]:
avgdf = df.filter(df["_3"]=="PM2.5").withColumn("dayavg", 
                                        (
                                        df["_4"]+df["_5"]+df["_6"]+df["_7"]+df["_8"]+df["_9"]+df["_10"]+df["_11"]\
                                        +df["_12"]+df["_13"]+df["_14"]+df["_15"]+df["_16"]+df["_17"]+df["_18"]+df["_19"]\
                                        +df["_20"]+df["_21"]+df["_22"]+df["_23"]+df["_24"]+df["_25"]+df["_26"]+df["_27"])/24)\
                                        .select("_1", "_2","dayavg")

In [ ]:
avgdf.show()

In [ ]:
avgdf.filter(avgdf.dayavg>60).show()

In [ ]:
avgdf.filter(avgdf.dayavg>60).groupby(avgdf._2).count().show()

In [ ]:
avgdf.filter(avgdf.dayavg>60)\
     .groupby(avgdf._2)\
     .count()\
     .orderBy('count', ascending=False)\
     .show()

# 使用 Spark SQL來下達SQL查詢

In [ ]:
df = spark.createDataFrame(dalipm25row)
df.show()
df.registerTempTable("Table")

In [ ]:
spark.sql("""
            select * from Table where date ='2015/02/07'
          """).show()

In [ ]:
s = spark.sql("""
                select count(*) count 
                from Table 
                where hr_01 > 100
               """)

In [ ]:
type(s)

In [ ]:
spark.sql("""
                select date, location 
                from Table
                where date="2015/02/07"
          """).show()

In [ ]:
spark.sql("""
                select date,location, hr_01, hr_02, hr_01-hr_02 as diff 
                from Table
          """).show()

## 練習1: 請使用SparkSQL，算算看2015全國哪個測站，紫爆天數最多？

In [ ]:
expr=""
for i in df.columns[1:25]:
  expr=str(i)+"+"+expr

print(expr)

In [ ]:
spark.sql("""
                select location, date, (hr_24+hr_23+hr_22+hr_21+hr_20+hr_19+hr_18+hr_17+hr_16+hr_15+hr_14+hr_13+hr_12+hr_11+hr_10+hr_09+hr_08+hr_07+hr_06+hr_05+hr_04+hr_03+hr_02+hr_01)/24 as avg
                from Table 
                having avg>60
          """).registerTempTable("DayAvgTable")

In [ ]:
spark.sql("""
          select location, count(date) as count
          from DayAvgTable
          group by location
          order by count DESC
          """).show()

# 利用DataFrame 直接讀取CSV, Fillna, DropNa功能清洗資料

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .getOrCreate()

from pyspark.sql import SQLContext
from pyspark.sql import Row

In [ ]:
df = spark.read.load("./pm25.csv", format="csv")

In [ ]:
df.show(200)

In [ ]:
df2=df.filter(df["_c0"]!="日期") 

In [ ]:
df2.count()

In [ ]:
df2.dropna()

In [ ]:
df2.count()

In [ ]:
df2.filter(df2._c3.isNotNull()).show()

In [ ]:
for i in df2.columns[3:]:
  df2 = df2.withColumn(i, df2[i].cast("double"))

In [ ]:
df2.dropna().count()

In [ ]:
df2 = df2.filter(df2["_c2"]!="RAINFALL")

# Perform Pearson Correlation using DataFrame.corr( )
# 練習: 計算大里區pm10, pm2.5 間之關聯度 http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame
使用前面所建立的clean_weather_data rdd資料
    
    
    corr(col1, col2, method=None)
    Calculates the correlation of two columns of a DataFrame as a double value. Currently only supports the Pearson Correlation Coefficient. DataFrame.corr() and DataFrameStatFunctions.corr() are aliases of each other.

    Parameters:	
    col1 – The name of the first column
    col2 – The name of the second column
    method – The correlation method. Currently only supports “pearson”
    New in version 1.4.

In [ ]:
def Generated_Measurement(x):
    date = x[0]
    location = x[1]
    measure = x[2]
    measurements_of_a_day = []
    for i, value in enumerate(x[3:]):
        measurements_of_a_day.append((date, measure, "hr_"+str(i), value))
    return measurements_of_a_day

daliData = clean_weather_data.filter(lambda x: x[1]=="大里" and (x[2] == "PM2.5" or x[2] == "PM10" ))
daliData.cache()

In [ ]:
daliData.toDF().show()

In [ ]:
from pyspark.sql import Row

daliDataRow = clean_weather_data.filter(lambda x: x[1]=="大里" and (x[2] == "PM2.5" or x[2] == "PM10" ))\
    .flatMap(Generated_Measurement)\
    .map(lambda x: ( (x[0], x[2]), x[1], x[3] ) )\
    .groupBy(lambda x: x[0])\
    .filter(lambda x: len(x[1])==2)\
    .mapValues(lambda x: list(x))\
    .mapValues(lambda x: [x[0][1], x[0][2], x[1][1], x[1][2]])\
    .map(lambda x:[ x[0][0], x[0][1], x[1][1], x[1][3]])\
    .map(lambda x: Row(
            date = x[0],
            time = x[1],
            pm10 = float(x[2]),
            pm25 = float(x[3])
        ))
    
df = spark.createDataFrame(daliDataRow)

In [ ]:
df.show(10)

In [ ]:
df.groupBy().avg().show()

In [ ]:
df.corr("pm10","pm25")

In [ ]:
df.describe().show()